## Testing Polynomials
We'll start with our imports as well as importing the rationals.

In [1]:
from groebner.polynomials import PolynomialRing, Polynomial

We can either instantiate `PolynomialRing` using the `num_vars` parameter or the `labels` parameter. If you provide labels, the `num_vars` parameter will be ignored.

In [2]:
# instantiating with custom labels
R = PolynomialRing(labels=['x','y','z'], base_field='QQ')
x, y, z = R.get_vars()

In [3]:
x + y

x + y

The next line will throw a warning because it does a coercion from a float to a rational. For small numbers it is safe to ignore, but you should still check that the polynomial you get out is correct. 

In [4]:
f = 3/4*x**2 + 2*y

/home/nico/documents/math/groebner/groebner/rationals.py:35: RuntimeWarning: Coercing to Rational from float is inherently inexact and will be incorrect for large numerators and denominators. If you need to have certainty the values are correct, you should directly instantiate with Rational(num, den) with integers.
  warn(


In [5]:
f

3/4x^2 + 2y

You can also ask for a "random" polynomial, optionally setting the degree and bound on the denominator and a bound on the degree (the position wrt the attached ordering).

In [6]:
R.random(num_terms=10, max_deg=20, denominator_bound=100)

1/5x^17y^20z^16 + 3/8x^19y^19z^12 + 53/54x^20y^19z^7 + 2/43x^17y^13z^13 + x^2y^19z^17 + 27/44x^19y^2z^13 + 10/13x^15yz^16 + 5/14x^3y^9z^20 + 44/61x^9z^14 + 1/3x^2y^7z^10

### Operations with polynomials
mostly work as you would hope

In [7]:
f*z**2

3/4x^2z^2 + 2yz^2

In [8]:
f**10

59049/1048576x^20 + 98415/65536x^18y + 295245/16384x^16y^2 + 32805/256x^14y^3 + 76545/128x^12y^4 + 15309/8x^10y^5 + 8505/2x^8y^6 + 6480x^6y^7 + 6480x^4y^8 + 3840x^2y^9 + 1024y^10

In [9]:
# coercion works
(x + y)*(x - y) + 1

x^2 - y^2 + 1

Using the other form of instantiation is more convenient because we don't have to bother writing out all the labels, but it can lead to some confusing results as below. Of course all of this can happen with bad variable naming.

In [10]:
S = PolynomialRing(num_vars=3, base_field='QQ')
x3, x2, x1 = S.get_vars()

In [11]:
x1

x_2

In [12]:
x1-x3**3

-x_0^3 + x_2

### Leading terms, coefficients, and monomials
Polynomials have the following helper methods mirroring $\text{LT},$ $\text{LM},$ and $\text{LC}$ from the book:

In [13]:
print(f.LC())
print(type(f.LC()))

3/4
<class 'groebner.rationals.Rational'>


In [14]:
print(f.LM())
print(type(f.LM()))

x^2
<class 'groebner.monomials.Monomial'>


In [15]:
print(f.LT())
print(type(f.LT()))

3/4x^2
<class 'groebner.polynomials.Polynomial'>


In [16]:
f.LT() == f.LC() * R.coerce(f.LM())

True

### Membership and types
These mostly work as you would hope.

In [17]:
x in R

True

In [18]:
2*x+3/4*y in R

True

In [19]:
x in R.field

False

In [20]:
type(2*x+3/4*y) is Polynomial

True

### Monomials and orderings
The core tool here is the `MonomialOrdering` class that gives us a way to well-order all monomials. Our implementation of `Polynomial` is basically just a list of coefficients and the ordering handles conversion between list index and monomial.

In [21]:
from groebner.monomials import MonomialOrdering, Monomial

In [22]:
# graded lexicographic ordering is the only one I have implemented.
o = MonomialOrdering(num_vars=3, labels=['x','y','w'], order_type='grlex')

Some examination of the insides of a monomial

In [23]:
m = Monomial([12, 32, 1], o)

In [24]:
m.degrees

[12, 32, 1]

In [25]:
m.total_degree

45

In [26]:
m.order == o

True

This is the level on which multiplication is implemented, so we can multiply two monomials (to get another monomial)

In [27]:
m*m

x^24y^64w^2

In [28]:
type(m*m) is Monomial

True

We also have comparison (using the ordering given by the ordering `o`)

In [29]:
s = Monomial([1,2,2], o)
t = Monomial([2,1,1], o)
print(s," || ",t)

xy^2w^2  ||  x^2yw


In [30]:
s > t

True